# Van der Pol equation

We consider the following problem :

\begin{equation} 
\left\{ 
\begin{aligned} 
{\mathrm d}_t x & = \varepsilon^2 \left(y-\frac{x^3}{3}+x \right)\\ 
{\mathrm d}_t y & = -x 
\end{aligned} 
\right. 
\qquad{} \text{with } \varepsilon > 0
\end{equation}


In [ ]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from ipywidgets import interact, IntSlider, Dropdown, FloatSlider, Text

from mylib.model import vanderpol_model, vanderpol_lienar_model
import mylib.integration as integration

output_notebook(hide_banner=True)

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 5 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
def plot_quasi_exact_sol():
    
    vdpm = vanderpol_lienar_model(eps=1)
    #vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn  
    
    tini = 0. 
    tend = 10.
    
    yini = (0.5, 0)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_sol = fig_sol.line(sol.t, sol.y[0], legend_label="x", line_width=2)    
    fig_sol.legend.location = "top_left"
    
    x = np.linspace(-3,3,1000)
    slow_manifold = x**3/3 - x
    
    fig_pp = figure(x_range=(-3, 3), y_range=(-3, 3), width=500, height=500, title="Phase portrait")
    plt_pp = fig_pp.line(x, slow_manifold, legend_label="slow manifold", line_width=2, color='red')   
    plt_pp = fig_pp.line(sol.y[0], sol.y[1], line_width=2)   
    fig_pp.xaxis.axis_label = "x"
    fig_pp.yaxis.axis_label = "y"
    
    show(column(fig_sol, fig_pp), notebook_handle=True)
    
    def update(eps) :
        vdpm = vanderpol_lienar_model(eps)
        #vdpm = vanderpol_model(eps)
        fcn = vdpm.fcn  
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
        plt_sol.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_pp.data_source.data = dict(x=sol.y[0], y=sol.y[1])
        push_notebook()
        
    interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=1., continuous_update=False))

    
plot_quasi_exact_sol()

## Characterization of stiffness

In [ ]:
def plot_eigen_values():

    vdpm = vanderpol_lienar_model(eps=1)
    #vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn  
    jac = vdpm.jac

    tini = 0. 
    tend = 10.
    
    yini = (0.5, 0)
    
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)
    
    eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
    for it in range(0,sol.t.size):
        eig_vals[it], _ = np.linalg.eig(jac(0., sol.y[:,it]))

    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]

    fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
    fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                      title = "Imaginary part of eigenvalues (click on legend entries to hide corresponding plot)")
    
    plt_real1 = fig_real.line(sol.t, np.real(lambda1), legend_label="lambda1")
    plt_real2 = fig_real.line(sol.t, np.real(lambda2), color="Green", legend_label="lambda2")

    plt_imag1 = fig_imag.line(sol.t, np.imag(lambda1), legend_label="lambda1")
    plt_imag2 = fig_imag.line(sol.t, np.imag(lambda2), color="Green", legend_label="lambda2")
    
    fig_real.legend.click_policy="hide"
    fig_imag.legend.click_policy="hide"

    show(column(fig_real, fig_imag), notebook_handle=True)
        
    def update(eps):
                
        vdpm = vanderpol_lienar_model(eps)
        #vdpm = vanderpol_model(eps)
        fcn = vdpm.fcn
        jac = vdpm.jac
                
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1.e-12, atol=1.e-12)

        eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
        for it in range(0,sol.t.size):
            eig_vals[it], _ = np.linalg.eig(jac(0., sol.y[:,it]))

        lambda1 = eig_vals[:, 0]
        lambda2 = eig_vals[:, 1]
        
        plt_real1.data_source.data = dict(x=sol.t, y=np.real(lambda1))
        plt_real2.data_source.data = dict(x=sol.t, y=np.real(lambda2))
        plt_imag1.data_source.data = dict(x=sol.t, y=np.imag(lambda1))
        plt_imag2.data_source.data = dict(x=sol.t, y=np.imag(lambda2))
        
        push_notebook()
 

    interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=1., continuous_update=False))

plot_eigen_values()

## Dormand and Price method

### Method of order 5

To compute global error, the quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 8 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
def plot_dopri5_sol():
    
    vdpm = vanderpol_lienar_model(eps=10)
    #vdpm = vanderpol_model(eps=10)
    fcn = vdpm.fcn  
    jac = vdpm.jac

    tini = 0. 
    tend = 10.
    
    yini = (0.5, 0)
    
    fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Time step")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")

    tol = 1.e-4
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    y_exa = integration.quasi_exact(fcn, yini, sol_dopri5.t)
    y1_err = np.abs(y_exa[:, 0] - sol_dopri5.y[0])

    plt_sol_x_y1 = fig_sol.x(sol_dopri5.t, sol_dopri5.y[0], legend_label="x", line_width=2)
    plt_sol_l_y1 = fig_sol.line(sol_dopri5.t, sol_dopri5.y[0], legend_label="x")    
    fig_sol.legend.location = "top_left"
    
    plt_err_y1 = fig_err.x(sol_dopri5.t, y1_err, line_width=2, legend_label="x")
    fig_err.legend.location = "bottom_left"
    
    dt = sol_dopri5.t[1::] - sol_dopri5.t[0:-1]
    plt_dt = fig_dt.x(sol_dopri5.t[0:-1], dt, line_width=2, legend_label="dt")
    
    show(column(fig_sol, fig_err, fig_dt), notebook_handle=True)
        
    def update(eps, tol):
        
        vdpm = vanderpol_lienar_model(eps)
        #vdpm = vanderpol_model(eps)
        fcn = vdpm.fcn
        jac = vdpm.jac

        sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
        y_exa = integration.quasi_exact(fcn, yini, sol_dopri5.t)
        y1_err = np.abs(y_exa[:, 0] - sol_dopri5.y[0])
        dt = sol_dopri5.t[1::] - sol_dopri5.t[0:-1]
        
        plt_sol_x_y1.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[0])
        plt_sol_l_y1.data_source.data = dict(x=sol_dopri5.t, y=sol_dopri5.y[0])
        plt_err_y1.data_source.data = dict(x=sol_dopri5.t, y=y1_err)
        plt_dt.data_source.data = dict(x=sol_dopri5.t[0:-1], y=dt)
        
        print("   Number of time step : " + str(sol_dopri5.t.size))

        push_notebook()
              
    dtol={'1.e-2':1.e-2, '1.e-4':1.e-4, '1.e-6':1.e-6, '1.e-8':1.e-8, '1.e-10':1.e-10}     
    interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=10., continuous_update=False), 
             tol=Dropdown(options=dtol, value=1.e-4, description='tol'))

plot_dopri5_sol()

### Method of order 8

To compute global error, the quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 8 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
def plot_dopri853_sol():
    
    vdpm = vanderpol_lienar_model(eps=10)
    #vdpm = vanderpol_model(eps=10)
    fcn = vdpm.fcn  
    jac = vdpm.jac

    tini = 0. 
    tend = 10.
    
    yini = (0.5, 0)
    
    fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
    fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Time step")

    tol = 1.e-4
    t_dopri853, y_dopri853 = integration.dopri853(tini, tend, yini, fcn, tol)
    y_exa = integration.quasi_exact(fcn, yini, t_dopri853)
    y1_err = np.abs(y_exa[:, 0] - y_dopri853[:,0])

    plt_sol_x_y1 = fig_sol.x(t_dopri853, y_dopri853[:, 0], legend_label="x", line_width=2)
    plt_sol_l_y1 = fig_sol.line(t_dopri853, y_dopri853[:, 0], legend_label="x")
    fig_sol.legend.location = "top_left"
    
    plt_err_y1 = fig_err.x(t_dopri853, y1_err, line_width=2, legend_label="x")
    fig_err.legend.location = "bottom_left"
    
    dt = t_dopri853[1::] - t_dopri853[0:-1]
    plt_dt = fig_dt.x(t_dopri853[0:-1], dt, line_width=2, legend_label="dt")
    
    show(column(fig_sol, fig_err, fig_dt), notebook_handle=True)
    
    def update(eps, tol):
        
        vdpm = vanderpol_lienar_model(eps)
        #vdpm = vanderpol_model(eps)
        fcn = vdpm.fcn
        jac = vdpm.jac

        t_dopri853, y_dopri853 = integration.dopri853(tini, tend, yini, fcn, tol)
        y_exa = integration.quasi_exact(fcn, yini, t_dopri853)
        y1_err = np.abs(y_exa[:, 0] - y_dopri853[:,0])
        dt = t_dopri853[1::] - t_dopri853[0:-1]

        plt_sol_x_y1.data_source.data = dict(x=t_dopri853, y=y_dopri853[:,0])
        plt_sol_l_y1.data_source.data = dict(x=t_dopri853, y=y_dopri853[:,0])
        plt_err_y1.data_source.data = dict(x=t_dopri853, y=y1_err)
        plt_dt.data_source.data = dict(x=t_dopri853[0:-1], y=dt)
        
        print("   Number of time step : " + str(t_dopri853.size))

        push_notebook()
        
    dtol={'1.e-2':1.e-2, '1.e-4':1.e-4, '1.e-6':1.e-6, '1.e-8':1.e-8, '1.e-10':1.e-10}     
    interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=10., continuous_update=False), 
             tol=Dropdown(options=dtol, value=1.e-4, description='tol'))
    
plot_dopri853_sol()